In [2]:
#30/7/24 Creates a K fold KNN model for each file in a folder provided they are Fingerprints

import os
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Fingerprints/Endocrine/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df.drop(columns=['SMILES'], inplace=True)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
finalresults = []
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Values'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        for mtry in range(1, 90):
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = KNeighborsClassifier(n_neighbors=mtry, weights='distance', 
                            algorithm='auto', leaf_size=100, p=2, metric='minkowski', 
                            metric_params=None, n_jobs=4)

            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = mtry
        #store best model for the given fold and plot the metric vs mcc value
        rf = KNeighborsClassifier(n_neighbors=bestmetrics, weights='distance', 
                            algorithm='auto', leaf_size=100, p=2, metric='minkowski', 
                            metric_params=None, n_jobs=4)
        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    sets += 1
    finalresults.append(results[12])

For dataset NR-AhRMACCSfp.csv
for fold 1 test set mcc of 0.6911651838917381 valid set mcc of 0.43208688018531766
for fold 2 test set mcc of 0.5774863796036351 valid set mcc of 0.5158261583396506
for fold 3 test set mcc of 0.6281224728184416 valid set mcc of 0.5351168830199021
for fold 4 test set mcc of 0.6504757059781684 valid set mcc of 0.5695194442276702
for fold 5 test set mcc of 0.7006347046760188 valid set mcc of 0.6191348181252644

validation metrics of:
positives in data 148
negatives in data 187
net accuracy = 0.7880597014925373
mcc = 0.5720642066959932
For dataset SR-MMPMACCSfp.csv
for fold 1 test set mcc of 0.6930271149175106 valid set mcc of 0.4316858359002737
for fold 2 test set mcc of 0.5871773243454872 valid set mcc of 0.5190499120887042
for fold 3 test set mcc of 0.45879607264416916 valid set mcc of 0.550350964762303
for fold 4 test set mcc of 0.5876681602870233 valid set mcc of 0.46061074593403173
for fold 5 test set mcc of 0.6529559667572015 valid set mcc of 0.48413653

for fold 4 test set mcc of 0.362030053499966 valid set mcc of 0.4740428361972617
for fold 5 test set mcc of 0.3889592577357173 valid set mcc of 0.488546503365251

validation metrics of:
positives in data 58
negatives in data 63
net accuracy = 0.6776859504132231
mcc = 0.40927265372891947
For dataset NR-ARAvafp.csv
for fold 1 test set mcc of 0.6572311106330365 valid set mcc of 0.33710193876865807
for fold 2 test set mcc of 0.6929407109164757 valid set mcc of 0.600245047998781
for fold 3 test set mcc of 0.48495047402467656 valid set mcc of 0.5570029454740558
for fold 4 test set mcc of 0.5290432684448219 valid set mcc of 0.2573206596865631
for fold 5 test set mcc of 0.5830006053876384 valid set mcc of 0.3572172541558801

validation metrics of:
positives in data 61
negatives in data 64
net accuracy = 0.76
mcc = 0.5589979253939279
For dataset SR-AREMorganfp.csv
for fold 1 test set mcc of 0.33743214149288236 valid set mcc of 0.36071493263878995
for fold 2 test set mcc of 0.3941755458745612 va

for fold 2 test set mcc of 0.4563376701880213 valid set mcc of 0.45726659240727496
for fold 3 test set mcc of 0.4595725150090289 valid set mcc of 0.4500265855315861
for fold 4 test set mcc of 0.40524896881636446 valid set mcc of 0.41518942736604736
for fold 5 test set mcc of 0.5184688572366651 valid set mcc of 0.16803478540247052

validation metrics of:
positives in data 73
negatives in data 79
net accuracy = 0.6447368421052632
mcc = 0.28755899927359346
For dataset SR-ATAD5MACCSfp.csv
for fold 1 test set mcc of 0.6264144833056663 valid set mcc of 0.3922322702763681
for fold 2 test set mcc of 0.4842969937928884 valid set mcc of 0.2311212814645309
for fold 3 test set mcc of 0.5013026509266503 valid set mcc of 0.3440511766019877
for fold 4 test set mcc of 0.4297371021269962 valid set mcc of 0.4767312946227962
for fold 5 test set mcc of 0.447213595499958 valid set mcc of 0.5270462766947299

validation metrics of:
positives in data 51
negatives in data 53
net accuracy = 0.75
mcc = 0.4998150


validation metrics of:
positives in data 61
negatives in data 64
net accuracy = 0.736
mcc = 0.5274949660495826
For dataset NR-aromataseTTorsionfp.csv
for fold 1 test set mcc of 0.43691762051643435 valid set mcc of 0.2118923641468466
for fold 2 test set mcc of 0.2600231530725279 valid set mcc of 0.5104035324589247
for fold 3 test set mcc of 0.5056205829602306 valid set mcc of 0
for fold 4 test set mcc of 0.32951276778634797 valid set mcc of 0.07993052538854532
for fold 5 test set mcc of 0.5005216395553902 valid set mcc of 0.6010910976500518

validation metrics of:
positives in data 58
negatives in data 63
net accuracy = 0.7024793388429752
mcc = 0.4646785977740792
For dataset SR-HSETTorsionfp.csv
for fold 1 test set mcc of 0.1829534767323246 valid set mcc of 0.18688841543883614
for fold 2 test set mcc of 0.41147167957154823 valid set mcc of 0.2077069959094179
for fold 3 test set mcc of 0.42763415500842855 valid set mcc of 0.2917148634810174
for fold 4 test set mcc of 0.4723929936340633 

In [1]:
def setbalance(df):
    import pandas as pd
    import numpy as np
    #df = df.iloc[:,1:]
    toxvals = df['Toxicity_Values']

    zercount = 0
    for value in toxvals:
        if value == 0:
            zercount += 1

    poscount = len(toxvals)-zercount
    domval = 0

    if poscount > zercount:
        temp = poscount
        poscount = zercount
        poscount = temp
        domval = 1

    if poscount*4 < zercount:
        ratio = zercount / poscount
        factor = int(ratio) - 1

        loops = 0
        todel = []
        for value in toxvals:
            if loops % factor != 0 and value == domval:
                todel.append(loops)
            loops += 1

        df = df.drop(todel)
        
    return df


In [2]:
namelist

['CarcinogenicityMorganfp.csv',
 'morganfp_SR-p53.csv',
 'morganfp_NR-AhR.csv',
 'morganfp_NR-ER-LBD.csv',
 'mutagenTTorsionfp.csv',
 'morganfp_NR-PPAR-gamma.csv',
 'Respiratory_ToxicityMorganfp.csv',
 'mutagenMACCSfp.csv',
 'Reproductive_ToxicityTTorsionfp.csv',
 'CarcinogenicityAvafp.csv',
 'CarcinogenicityTTorsionfp.csv',
 'CarcinogenicityMACCSfp.csv',
 'morganfp_NR-ER.csv',
 'Respiratory_ToxicityAvafp.csv',
 'Developmental_ToxicityTTorsionfp.csv',
 'morganfp_SR-ATAD5.csv',
 'Developmental_ToxicityMorganfp.csv',
 'mutagenMorganfp.csv',
 'mutagenAvafp.csv',
 'Reproductive_ToxicityAvafp.csv',
 'morganfp_SR-ARE.csv',
 'Developmental_ToxicityAvafp.csv',
 'morganfp_NR-AR.csv',
 'Reproductive_ToxicityMorganfp.csv',
 'Developmental_ToxicityMACCSfp.csv',
 'morganfp_SR-HSE.csv',
 'morganfp_SR-MMP.csv',
 'Respiratory_ToxicityTTorsionfp.csv',
 'Respiratory_ToxicityMACCSfp.csv',
 'morganfp_NR-aromatase.csv',
 'morganfp_NR-AR-LBD.csv',
 'Reproductive_ToxicityMACCSfp.csv']

In [15]:
temp = pd.DataFrame(finalresults, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Data/Model_Outputs/Fingerprint/KNN_Fingerprint_Results')

In [14]:
resultframe

,Dataset,MCC
0,CarcinogenicityMorganfp.csv,0.300006
1,morganfp_SR-p53.csv,0.392793
2,morganfp_NR-AhR.csv,0.572339
3,morganfp_NR-ER-LBD.csv,0.453839
4,mutagenTTorsionfp.csv,0.514638
5,morganfp_NR-PPAR-gamma.csv,0.353500
6,Respiratory_ToxicityMorganfp.csv,0.421679
7,mutagenMACCSfp.csv,0.614949
8,Reproductive_ToxicityTTorsionfp.csv,0.000000
9,CarcinogenicityAvafp.csv,0.269629
